# United States

## Unused Doses Methodology

To calculate unused doses by a certain date, we take the following steps:

 1. Estimate how many doses the country has received (and/or will) by a specific date
 2. Estimate how many doses have been used (and/or will) by a specific date, should vaccinations continue at the same rate as now (latest 14-day period)
 3. Estimate how many doses the country has donated (and/or will, based actual or projected deliveries) by a specific date.
 4. Estimate how many unused or excess doses there would be by a specific date, calculated as:<br> 
 
$Unused_{doses} = Supply - Demand - Donations$

### Prep

Before starting the analysis, the necessary libraries and functions need to be imported. To keep this section clean, most of the scripts are stored in a file called [`utils.py`](https://github.com/ONEcampaign/project_covid-19_tracker/blob/v2/stories/supply_demand/utils.py)

In [1]:
import os
import sys
project_path = os.path.dirname(os.path.abspath(os.path.join('')))
scripts_path = os.path.dirname(os.path.abspath(os.path.join('/jupyter_notebooks/')))

if project_path not in sys.path:
    sys.path.append(project_path)

#Import pandas and the utilities    
import pandas as pd
from jupyter_notebooks import utils


#Format the display of numbers
from IPython.display import display
pd.options.display.float_format = '{:,.0f}'.format


def show(title=None, data=None):
    if title is not None:
        title = "\033[1m" + title + "\033[0m"
    print('\n',title)
    display(data)
    
def number(number):
    return "{:,.0f}".format(number)



### Step 1: Supply

In [2]:
#US Received Supply Data
supply_received = utils.get_supply(filename='us_received_airfinity')

#US Forecast Supply Data. 
vaccines = ['Johnson&Johnson', 'Pfizer/BioNTech', 'Moderna']
supply_forecast = utils.get_supply_forecast(filename='us_supply_forecast', 
                                            vaccines = vaccines)

#Add supply change to forecast
supply_forecast = utils.add_supply_change_to_forecast(supply_forecast)

#Forecast summary
supply_forecast_summary = utils.additional_supply_by_month(supply_forecast)


#Store key numbers for further analysis
current_supply_delivered = supply_received.delivered_supply.sum()
supply_forecast_year_end = supply_forecast_summary['Total'].sum()


#Show summary
supply_forecast_short = supply_forecast_summary.query("date == 'August' or date== 'December'")
supply_received_short = supply_received.query('delivered_supply > 0')

print(f'The US has received {number(current_supply_delivered)} doses')
print(f'It will have received {number(supply_forecast_year_end)} doses by the end of the year.')


show('Supply Forecast (additional doses per month)',supply_forecast_short)
show('Supply received to date',supply_received_short)



The US has received 390,090,255 doses
It will have received 948,506,615 doses by the end of the year.

 Supply Forecast (additional doses per month)


,country,date,Johnson&Johnson,Moderna,Pfizer/BioNTech,Total
9,United States,August,"15,152,287","33,803,191","30,249,422","79,204,900"
13,United States,December,"21,258,183","32,712,766","60,402,315","114,373,264"



 Supply received to date


,country,vaccine,delivered_supply,confirmed_supply,supply_date
0,United States,Johnson&Johnson,"21,484,400","200,000,000",2021-07-18
1,United States,Moderna,"161,006,080","410,000,000",2021-07-18
4,United States,Pfizer/BioNTech,"207,599,775","500,000,000",2021-07-18


### Step 2: Demand

This demand analysis is based on current trends up to 18-July, based on the demand of the previous 14 days period. It assumes a linear trend that continues indefinitely.

Data is available for the US in order to calculate the demand by vaccine type.

In [3]:
#Get demand data (from OWID), clean and add linear projections with 14-day window

demand = (utils.get_used_doses_by_manufacturer(vaccines)
          .pipe(utils.clean_used_doses_by_manufacturer)
          .pipe(utils.linear_projection_by_manufacturer,  observed_max_date='2021-07-27',days_window=14)
          .pipe(utils.add_change, 'cumulative_doses','additional_doses')
         )

#Filter the demand data for the US
us_demand = demand.query("iso_code == 'USA'")

#Show a summary of the forecast demand data
demand_summary = utils.additional_used_by_month(us_demand, vaccines)

show('Observed/forecast Demand by month', demand_summary)

print(f'Total expected demand this year: {number(demand_summary.Total.sum())} doses')



 Observed/forecast Demand by month


,iso_code,date,Johnson&Johnson,Pfizer/BioNTech,Moderna,Total
0,USA,January,0,"16,775,916","14,246,089","31,022,005"
1,USA,February,0,"21,650,651","22,448,325","44,098,976"
2,USA,March,"3,307,113","38,465,345","33,236,542","75,009,000"
3,USA,April,"4,924,026","50,241,487","34,682,937","89,848,450"
4,USA,May,"2,545,478","32,278,632","19,907,021","54,731,131"
5,USA,June,"1,629,129","20,932,966","8,993,954","31,556,049"
6,USA,July,"975,405","12,211,590","5,105,944","18,292,939"
7,USA,August,"865,204","11,758,182","4,463,667","17,087,053"
8,USA,September,"837,295","11,378,886","4,319,677","16,535,858"
9,USA,October,"865,204","11,758,182","4,463,667","17,087,053"


Total expected demand this year: 428,891,423 doses


In [4]:
#The analysis also requires estimates on full vaccinations per month
full_vaccinations = (utils.get_owid_vaccination()
                     .pipe(utils.full_vaccination_forecast, observed_max_date='2021-07-27'))

#Filter for US only
us_full_vaccinations = full_vaccinations.query("iso_code == 'USA'")

us_full_vaccinations_summary = utils.additional_vax_by_month(us_full_vaccinations)
us_full_vaccinations_summary['Fully_vaccinated_cumulative'] = us_full_vaccinations_summary.additional_fully_vaxxed.cumsum()

#Show key numbers
show('Observed / Forecast Full Vaccionations (additional per month)', us_full_vaccinations_summary)
print(f'An average of {number(us_full_vaccinations_summary.iloc[7:,2].mean())} full vaccinations per month (Aug-Dec)')



 Observed / Forecast Full Vaccionations (additional per month)


,iso_code,date,additional_fully_vaxxed,Fully_vaccinated_cumulative
0,USA,January,"5,657,142","5,657,142"
1,USA,February,"19,122,778","24,779,920"
2,USA,March,"29,827,121","54,607,041"
3,USA,April,"46,800,277","101,407,318"
4,USA,May,"33,680,001","135,087,319"
5,USA,June,"19,797,367","154,884,686"
6,USA,July,"9,593,427","164,478,113"
7,USA,August,"7,928,734","172,406,847"
8,USA,September,"7,672,968","180,079,815"
9,USA,October,"7,928,734","188,008,549"


An average of 7,826,428 full vaccinations per month (Aug-Dec)


### Step 3: Donations

This section contains donations committed and delivered as of 18 July, according to Airfinity's data. It is possible that it does not include all doses effectively delivered in July. 

Our forecast for August assumes that all the remaining doses committed (outside the 500m doses of Pfizer) are delivered that month.

In [5]:
#Load the US donations to date

us_donations = utils.get_donations_data(filename='us_donations_airfinity')
us_donations_summary = utils.donations_to_date_summary(us_donations)

donations_to_date =us_donations_summary.copy()
donations_to_date['date']='July'
donations_to_date = donations_to_date.groupby('date').sum()['delivered'].reset_index(drop=False)
donations_to_date.rename(columns= {'delivered':'Total_Donations'}, inplace=True)

us_donations_summary['Cumulative_delivered'] = us_donations_summary.delivered.cumsum()

#Donations forecast
doses = {'Pfizer/BioNTech': {'August': 3.5*1e6,'September': 50*1e6,'October': 50*1e6,'November': 50*1e6,'December': 50*1e6},
         'Moderna':         {'August': 2168000,'September': 0,'October': 0,'November': 0,'December': 0},
         'Johnson&Johnson': {'August': 1.5*1e6,'September': 0,'October': 0,'November': 0,'December': 0},
         'Oxford/AstraZeneca': {'August': 0, 'September': 0,'October': 0,'November': 0,'December': 0},
         }
us_forecast_donations = utils.us_donation_schedule(doses)
us_forecast_donations_summary = utils.donations_forecast_summary(us_forecast_donations)
us_forecast_donations_summary.rename(columns={'Total': 'Total_Donations'}, inplace=True)



#Show key figures
show('US Donations Summary (observed)', us_donations_summary)
show('US Donations Forecast', us_forecast_donations_summary)

total_donations_2021 = us_donations_summary.delivered.sum() + us_forecast_donations_summary.Total_Donations.sum()
print(f'A total of {number(total_donations_2021)} (delivered + forecasted) donated doses in 2021')


 US Donations Summary (observed)


,vaccine,donated,delivered,delivery_date_last,Cumulative_delivered
0,Johnson&Johnson,"7,518,000","5,350,000",2021-07-12,"5,350,000"
1,Moderna,"13,500,000","12,000,000",2021-07-18,"17,350,000"
2,Pfizer/BioNTech,"207,500,480","4,000,480",2021-07-15,"21,350,480"



 US Donations Forecast


,date,Johnson&Johnson,Moderna,Oxford/AstraZeneca,Pfizer/BioNTech,Total_Donations,Cumulative
0,August,"1,500,000","2,168,000",0,"3,500,000","7,168,000","7,168,000"
1,September,0,0,0,"50,000,000","50,000,000","57,168,000"
2,October,0,0,0,"50,000,000","50,000,000","107,168,000"
3,November,0,0,0,"50,000,000","50,000,000","157,168,000"
4,December,0,0,0,"50,000,000","50,000,000","207,168,000"


A total of 228,518,480 (delivered + forecasted) donated doses in 2021


### Step 4: Unused doses

In [6]:
#At any point, unused is supply minus demand minus donations
#Create table with all aggregate numbers

df = (supply_forecast_summary[['country','date','Total']]
      .merge(demand_summary[['date','Total']], on=['date'], how='left', suffixes=('_Supply','_Demand'))
      .merge(donations_to_date.append(us_forecast_donations_summary[['date','Total_Donations']],ignore_index=True),
             on=['date'], how='left')
     )
df['Unused'] = df.Total_Supply - df.Total_Demand.fillna(0) - df.Total_Donations.fillna(0)

df_summary = df.groupby('country').sum().reset_index(drop=False)

df['Unused Cumulative'] = df.Unused.cumsum()


df_short = df.query('date=="August" or date == "December"')

show('Unused doses analysis for the US', df_short.fillna(0))

show('As a result, by the end of the year, the situation will look like this',df_summary)


 Unused doses analysis for the US


,country,date,Total_Supply,Total_Demand,Total_Donations,Unused,Unused Cumulative
9,United States,August,"79,204,900","17,087,053","7,168,000","54,949,848","149,395,585"
13,United States,December,"114,373,264","17,087,053","50,000,000","47,286,211","291,096,712"



 As a result, by the end of the year, the situation will look like this


,country,Total_Supply,Total_Demand,Total_Donations,Unused
0,United States,"948,506,615","428,891,423","228,518,480","291,096,712"


### Export analysis

In [7]:
#Export
data_to_export = {'Supply Forecast': supply_forecast_summary,
                  'Demand Summary': demand_summary,
                  'Observed/Forecast Full Vaccinations (Additional per month)': us_full_vaccinations_summary,
                  'US donations forecast': us_forecast_donations_summary,
                  'US Unused Doses Analysis': df}

utils.export_excel(file_name= 'US_Supply and Demand analysis',
                   worksheet_name = 'US_analysis',
                   dataframes_dict = data_to_export)